<a href="https://colab.research.google.com/github/chhyyi/aiffel/blob/main/exp6/AIFFEL_LMS_EXP6_Lyricist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lyricist
Aiffel LMS exploration node 6 (by modulabs), lyricist 만들기

## Step1 데이터 다운로드, step2 데이터 불러오기
- 데이터셋을 불러오기 위해 구글 드라이브 마운트합니다.
- 미리 다운로드를 받아서 구글 드라이브에 저장해두었습니다. colab으로 매번 옮기기가 번거롭기는 하지만 공유하기가 이쪽이 편하니 여기서 하겠습니다.  
- 버전을 확인해보면 2.8.2.가 나오는데 큰 차이가 없기를 기대해 봅니다.

In [ ]:
!ls /content/drive/MyDrive/colabdata/modulabs/lms_exp6

adele.txt	       dolly-parton.txt        ludacris.txt
al-green.txt	       drake.txt	       michael-jackson.txt
alicia-keys.txt        dr-seuss.txt	       missy-elliott.txt
amy-winehouse.txt      eminem.txt	       nickelback.txt
beatles.txt	       janisjoplin.txt	       nicki-minaj.txt
bieber.txt	       jimi-hendrix.txt        nirvana.txt
bjork.txt	       johnny-cash.txt	       notorious_big.txt
blink-182.txt	       joni-mitchell.txt       notorious-big.txt
bob-dylan.txt	       kanye.txt	       nursery_rhymes.txt
bob-marley.txt	       kanye-west.txt	       patti-smith.txt
britney-spears.txt     Kanye_West.txt	       paul-simon.txt
bruce-springsteen.txt  lady-gaga.txt	       prince.txt
bruno-mars.txt	       leonard-cohen.txt       radiohead.txt
cake.txt	       lil-wayne.txt	       rihanna.txt
dickinson.txt	       Lil_Wayne.txt	       r-kelly.txt
disney.txt	       lin-manuel-miranda.txt
dj-khaled.txt	       lorde.txt


In [ ]:
import tensorflow as tf
tf.__version__

'2.8.2'

In [ ]:
import glob

txt_file_path = '/content/drive/MyDrive/colabdata/modulabs/lms_exp6/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 186966
Examples:
 ['Jolene, Jolene, Jolene, Jolene', "I'm begging of you please don't take my man", 'Jolene, Jolene, Jolene, Jolene']


## Step 3. 데이터 정제
토큰 개수가 15개를 넘는 문장은 제외하라고 권장하고 있습니다. preprocess_sentence()를 사용하라고 하므로 일단 함수를 가져옵니다. 함수에 대한 코멘트도 그대로 가져왔습니다.
> 입력된 문장을
1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 \<start>, 끝에는 \<end>를 추가합니다  
이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!

In [ ]:
import re 
import numpy as np

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [ ]:
print(preprocess_sentence("L-O-V-E-L-E-S-S"))

<start> l o v e l e s s <end>


바로 반복되는 문장도 지워버렸습니다. 문장 단위의 sequence는 어차피 안보니까 상관 없다고 판단했습니다.  
띄어쓰기 15개 이상으로 구분되는 string도 지웁니다. 토큰화 했을 때 지우라고 했는데 그냥 여기서 처리했습니다.

In [ ]:
corpus = []
dummy=''

for sentence in raw_corpus:
    if len(sentence) == 0 or dummy==sentence: continue #바로 반복되는 같은 문장도 지워버렸습니다.
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split(sep=' '))>15: continue #띄어쓰기 15개 이상으로 구분되는 string도 지웁니다.
    corpus.append(preprocessed_sentence)
    dummy=sentence
        
#check up of refined data
corpus[:10]

['<start> jolene , jolene , jolene , jolene <end>',
 '<start> i m begging of you please don t take my man <end>',
 '<start> jolene , jolene , jolene , jolene <end>',
 '<start> with flaming locks of auburn hair <end>',
 '<start> with ivory skin and eyes of emerald green <end>',
 '<start> your smile is like a breath of spring <end>',
 '<start> your voice is soft like summer rain <end>',
 '<start> and i cannot compete with you <end>',
 '<start> jolene he talks about you in his sleep <end>',
 '<start> and there s nothing i can do to keep <end>']

해석 불가능한 가사가 나와서 조금 살펴봅니다.
- it's, He's 를 it s, He s와 같이 두 토큰으로 변환하는 것이 괜찮을까요? 큰 문제는 없을 것 같습니다.
- 일부 파일에서 제목으로 보이는 것들이 발견됩니다만 일단 그냥 두겠습니다.
- 반복되는 문장이 상당히 많습니다.
- L-O-V-E-L-E-S-S 는 괜찮을까요? 이건 l o v e l e s s 8개의 토큰이 될 것 같습니다.
- Bruno Mars의 경우 일부 곡에 스페인어를 가사에 쓴 것 같습니다. 일단 그 부분만 임의로 삭제하겠습니다. 그냥 파일에서 지웠습니다.  
빈 줄과 반복되는 줄, 15개가 넘는 띄워쓰기로 구분된 문장을 지운 결과 raw_corpus와 corpus의 길이가 187088, 151932으로 만큼 차이가 납니다.

In [ ]:
print(len(corpus), len(raw_corpus))

151825 186966


## Step4. 평가 데이터셋 분리
단어장의 크기는 12000이상으로 하라고 하니까 13000으로 했습니다. 나머지는 \<unk>라고 해버립니다. padding은 길이를 맞춰주는 겁니다.

In [ ]:
#copied code from lms
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=13000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 1289    5 ...    0    0    0]
 [   2    4   25 ...    3    0    0]
 [   2 1289    5 ...    0    0    0]
 ...
 [   2   48   48 ...    0    0    0]
 [   2    4   25 ...    0    0    0]
 [   2   48   48 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa8b8204c50>


In [ ]:
#copied code from lms
src_input = tensor[:, :-1] #source에서 <end>를 제거
tgt_input = tensor[:, 1:] #target에서 <start>를 제거

print(src_input[0])
print(tgt_input[0])

[   2 1289    5 1289    5 1289    5 1289    3    0    0    0    0    0]
[1289    5 1289    5 1289    5 1289    3    0    0    0    0    0    0]


In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=32)

In [ ]:
#from copied code from lms, make two dataset of train and validation.
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)

BUFFER_SIZE = len(dec_train)
steps_per_epoch = len(dec_train) // BATCH_SIZE

dataset_eval = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_eval = dataset_eval.shuffle(BUFFER_SIZE)
dataset_eval = dataset_eval.batch(BATCH_SIZE, drop_remainder=True)
print('\n dataset_eval')
print(dataset_eval)

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

 dataset_eval
<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>


## step5. 인공지능 만들기
> 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!

이번에도 6-5의 코드를 가져와서 만들어보겠습니다.




In [ ]:
#copied code from lms
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

모델을 확인하기 위해 데이터셋에서 하나만 뽑아서 넣어봅니다.

In [ ]:
#copied code from lms
for src_sample, tgt_sample in dataset_train.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 13001), dtype=float32, numpy=
array([[[ 4.4230157e-05,  6.3468593e-05,  1.3292658e-04, ...,
         -6.3161977e-05, -2.6610674e-04,  4.5868641e-05],
        [-6.4160943e-04,  4.6776913e-04,  1.1165556e-03, ...,
         -2.1839657e-04, -5.6659634e-04,  5.1304145e-04],
        [-1.1225600e-03,  6.5918185e-04,  1.7859292e-03, ...,
         -3.8471975e-04, -7.5735041e-04,  3.0967922e-04],
        ...,
        [-1.8769091e-04, -2.3470669e-04,  2.3774614e-03, ...,
         -1.1077181e-03, -8.1038492e-04,  2.3716381e-03],
        [ 4.5113734e-04, -5.9437245e-04,  3.6216537e-03, ...,
         -6.6630135e-04, -1.3476746e-04,  3.4521229e-03],
        [ 1.0500229e-03, -9.0237206e-04,  4.8800646e-03, ...,
         -2.3464498e-04,  5.6878943e-04,  4.4592666e-03]],

       [[ 4.4230157e-05,  6.3468593e-05,  1.3292658e-04, ...,
         -6.3161977e-05, -2.6610674e-04,  4.5868641e-05],
        [ 1.1420773e-05,  4.5288651e-04,  3.9101724e-04, ...,
         -1.2631807e-04, 

In [ ]:
#copied code from lms

model.summary()
tf.test.is_gpu_available()

Model: "text_generator_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  13313024  
                                                                 
 lstm_6 (LSTM)               multiple                  25174016  
                                                                 
 lstm_7 (LSTM)               multiple                  33562624  
                                                                 
 dense_3 (Dense)             multiple                  26639049  
                                                                 
Total params: 98,688,713
Trainable params: 98,688,713
Non-trainable params: 0
_________________________________________________________________


True

In [ ]:
#copied code from lms

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

model.fit(dataset_train, epochs=6, validation_data=(enc_val,dec_val))

Epoch 1/6
474/474 [==============================] - 143s 297ms/step - loss: 3.2754 - val_loss: 2.9106
Epoch 2/6
474/474 [==============================] - 139s 292ms/step - loss: 2.7379 - val_loss: 2.6678
Epoch 3/6
474/474 [==============================] - 139s 292ms/step - loss: 2.4218 - val_loss: 2.4935
Epoch 4/6
474/474 [==============================] - 139s 292ms/step - loss: 2.1067 - val_loss: 2.3568
Epoch 5/6
474/474 [==============================] - 139s 292ms/step - loss: 1.8067 - val_loss: 2.2545
Epoch 6/6
474/474 [==============================] - 138s 292ms/step - loss: 1.5382 - val_loss: 2.1955


## val_loss의 평가
위에서 나오는 loss는 train data에 대한 것이고 validation data에 대한 loss를 구해봅니다.

In [ ]:
model.metrics_names

['loss']

In [ ]:
eval_loss = model.evaluate(enc_val,dec_val)
print('eval_loss: {} '.format(eval_loss))

949/949 [==============================] - 21s 22ms/step - loss: 2.1955
eval_loss: 2.1955366134643555 


In [ ]:
print('',embedding_size, hidden_size,'', eval_loss,'', sep='|') #markdown 표에 붙여놓기 쉽게 해봅니다. 그런데 train_loss랑 step당 걸리는 시간은 못찾겠네요.

|1024|2048||2.1955366134643555|


이 단계까지를 반복하면서 적당한 embedding, hidden size를 찾아서 10 epoch 안에 val_loss를 2.2 밑으로 줄여보라고 합니다. 그러므로 epoch = 10의 결과를 간단히 정리하겠습니다.
*처음에 실수로 30epoch까지 돌렸던 결과입니다. overfitting이 됐다고 짐작할 수 있을 것 같습니다. train_loss는 안적어놔서 모르는데 0.9정도 였던 거 같습니다.  
embedding size와 hidden size를 늘리기 시작하자, train_loss가 굉장히 작아지기 시작했습니다. 이것은 overfitting 되고 있다는 뜻일까요?

|embedding size|hidden size|train_loss|eval_loss|time/step|epochs|
|-----|----|----|---|---|---|
|256|1024|?|*2.6430249214172363|?|30|
|256|1024|2.2778|2.579704999923706|?|10|
|128|512|2.6172|2.7422351837158203|?|10|
|512|2048|1.2204|2.21757173538208|240ms|10|
|1024|2048|0.9981|2.2050082683563232|252ms|10|
|512|4096|1.0186|2.238615036010742|741ms|10|
|1024|1024|2.0810|2.481119155883789|104ms|10|
|512|2048|2.2549|2.484931707382202|241ms|5|
|512|2048|1.6797|2.3204803466796875|241ms|8|
|512|2048|1.3387|2.2849111557006836|241ms|10|
|1024|2048|1.7487|2.2373404502868652|252ms|5|

embedding size 1024, hidden size 2048인 경우에 대해서 집중하겠습니다.  

|epochs|train_loss|eval_loss|  
|----|----|----|  
|5|1.7487|2.2373404502868652|
|6|1.5201|2.1752|
|7|1.2722|2.1769|
|8|1.1286|2.1950|
|9|1.0452|2.2182|
|10|1.0023|2.2417|

여기서 overfitting이 일어나고 있는 것 같습니다.
epoch 7인 경우에서 작사를 시켜보겠습니다.





In [ ]:
# copied from lms node
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> Say")

'<start> say it all say it all , <end> '

# 회고

## 런타임 유형과 나의 과거
수동으로 grid search와 같은 일을 엄청나게 해버렸다. 처음으로 꽤 긴 시간이 걸리는 계산을 돌려야 했다. 옛날 기억들이 떠올랐다. 연구실 자리에서, 나는 바로 뒤에 있는 컴퓨터에 원격으로 접속해서 이런저런 계산을 돌리고 그 결과를 기다리며 할 다른 일을 찾아야 했다. 그래서 게임을 많이 했다.  
 나는 이 노트북을 실행시키기 위해 colab 노트북의 런타임 유형을 처음으로 변경해야 했다. 지금까지 자동으로 변경됐던 경우도 있었을까? 결국 계산자원(computing power)에 대한 문제가 남아있다. 얼마 전에 아는 사람에게 듣자하니 구글에서는 TPU research cloud를 무료로 사용하게 해주는 프로그램을 진행하고 있다고 한다. 회사나 단체 등에서 지원하면 심사해서 쓰게 해주는 것 같다. 왜 그런 걸 진작에 알지 못했을까?   
참 생각이 많아지는 문제다. 어떤 아는 사람에게 캐글 competition에서 돈을 벌 수 있냐고 묻자 '상위권 가려면 컴퓨팅 파워가 좋아야 한다'라고 답했다.  
짧은 경험이지만 계산용 컴퓨터를 관리하는 것은 끔찍했다. 나는 결국 GPU를 붙이자는 본래 계획을 폐기하고 상당부분을 부품 교체에 써야 했다. 계산용으로 128GB램에 제온 cpu를 붙여서 구매한 그 녀석은 채 1년이 지나 삐걱거리기 시작했고, 나는 결국 보통 PC처럼 껏다 켰다 하면서 써야 했다. 그런데 계획대로 GPU를 썼을 때 얼마나 성능향상이 있을지는 전혀 알 수 없는 상태였기 때문에 오히려 다행이었는지도 모르겠다.  

### 단어장의 크기 등
 이 부분은 12000이상으로 하라고 하여서 13000으로 그냥 고정시켰는데 이게 맞을까? 다르게 했다면 어땠을까? 또 중복문장을 제거하는 것이 의미가 있는 처리였을까? 가사 일부를 지우는 등 임의로 처리한 부분도 있는데 이것은 어떨까? 노래 가사라면 좀 모르는 말이 갑자기 나와도 괜찮다고 봐야 할까?

## 중간과정에서 가사를 참조하지 않음
중간과정에서 이 model이 어떤 문장을 내뱉을 것인지는 전혀 생각하지 않았다. lms노드에서는 이런 model의 평가방법 중 하나로 '그럴싸한 문장을 만드는지 사람이 보고 평가'하는 것을 제시하고 있었지만 전혀 고려하지 않았따. 문득 처음 작사가 모델을 만들 거라고 했을 때 멜로디가 있냐고 물었던 것이 기억난다. 만약 가사와 멜로디가 붙어있다면, 아주 간단하게는 음의 높이와 길이만 주어져 있다고 해도 단어를 다시 음절로 분리할 필요가 있었을 것이다. 아무튼 각각의 발음과 음절의 길이에 의해 형성되는 운율감을 고려하게 됐을 것이고, 그러면 좀 더 흥미로웠겠지만 훨씬 무거운 계산이 되었을 것이다. 내가 say라고 넣자 이 모델은 'say it all say it all ,' 이라는 결과를 내놓았다. 어떤 장르에라도 나올 법한 가사이다. 그래서 템포나 멜로디 같은 것이 떠오르지는 않았다. 만약 특정 장르의 가사만을 뽑아서 넣었다면 좀 달랐겠지?